In [1]:
import pandas as pd
import subprocess
import math
import glob
import os, sys
from collections import defaultdict, Counter, OrderedDict
from Bio import SeqIO, SeqRecord, Seq
from tRNA_position import *
pd.set_option('display.max_colwidth',10000)
pd.set_option('display.width', 10000)
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows',1000)
isotypes = ['Ala', 'Arg', 'Asn', 'Asp', 'Cys', 'Gln', 'Glu', 'Gly', 'His', 'Ile', 'iMet', 'Leu', 'Lys', 'Met', 'Phe', 'Pro', 'Ser', 'Thr', 'Trp', 'Tyr', 'Val']

## Introduction

A global view of identity elements versus biological features would be a powerful tool for predicting tRNA function using primary sequence. To do this, I'll need to align eukaryotic tRNAs to our established models and annotate positions based on universal numbering. Then, I'll extract sequence information such as position, clade, or variable arm length.

## Process tRNAs
### Species information

In [2]:
species_table = pd.read_table('genome_table+.txt', header=None, names=['shortname', 'name', 'longname', 'domain', 'clade', 'subclade'])
# set NaN to empty strings
species_table.ix[species_table.name.apply(lambda x: type(x) is float), 'name'] = ''
species_table.head(20)

,shortname,name,longname,domain,clade,subclade
0,Bdist3,braDis3,Brachypodium distachyon Bd21 (JGI v3.0),eukaryota,Streptophyta,NaN
1,Mtrun4,medTru4,Medicago truncatula (Mt4.0v1),eukaryota,Streptophyta,NaN
2,Graim2,gosRai2,Gossypium raimondii (Cotton v2),eukaryota,Streptophyta,NaN
3,Ptric3,popTri3,Populus trichocarpa (JGI v3.0),eukaryota,Streptophyta,NaN
4,Sbico3,sorBic3,Sorghum bicolor (Version 3.1),eukaryota,Streptophyta,NaN
5,Oarie1,oviAri3,Ovis aries (sheep) (Feb 2010),eukaryota,Mammalia,NaN
6,Zmays6,zeaMay6,Zea mays B73 (Version AGPv3),eukaryota,Streptophyta,NaN
7,Gmax2,glyMax2,Glycine max (soybean) (Wm82.a2),eukaryota,Streptophyta,NaN
8,Rnorv6,rn6,Rattus norvegicus (Rat Jul. 2014 RGSC 6.0/rn6),eukaryota,Mammalia,NaN
9,Vpaco2,vicPac2,Vicugna pacos (Alpaca Mar. 2013 Vicugna_pacos-2.0.1/vicPac2),eukaryota,Mammalia,NaN


### Align tRNAs to existing CMs

We have a "quality set" of tRNAs, which include tRNAs from previous versions of genomes and newer versions of genomes. It is simpler and easier to create a new fasta file and alignment for each isotype, based on the `.iso` files from each run.

I also need to manually curate the genome table file to include the new species. And while I'm doing that, I might as well fix the broken entries.

In [39]:
species = sorted(glob.glob("iso/*.iso"))
species = [sp[4:-10] for sp in species]
isotype_seqs = defaultdict(list)
for sp in species:
  sys.stdout.write('processing ' + sp + '...')
  sys.stdout.flush()
  tRNA_file = 'tRNAs/' + sp + '-tRNAs.fa'
  if not os.path.exists(tRNA_file):
    print(sp + ' tRNA file not found, skipping')
    continue

  seqs = []
  seqids = []
  for seq in SeqIO.parse(tRNA_file, 'fasta'):
    if "pseudogene" in seq.description: continue
    score = float(re.findall('Sc: [\d\.]+', seq.description)[0].split()[-1])
    if score < 55: continue
    seqs.append(seq)
    seqids.append(seq.id)

  iso_scores = pd.read_table('iso/' + sp + '-tRNAs.iso', header=0)
  iso_scores = iso_scores[iso_scores.tRNAscanID.isin(seqids)]
  iso_scores['best'] = iso_scores.ix[:,2:].idxmax(axis=1)
  iso_scores['score'] = iso_scores.max(axis=1, numeric_only=True)
  iso_scores.index = iso_scores.tRNAscanID.values
  iso_scores = iso_scores[['best', 'score']]
  
  for seq in seqs:
    isotype = iso_scores.ix[seq.id].best
    if isotype == 'SeC' or 'mito' in isotype: continue
    isoscore = iso_scores.ix[seq.id].score
    seq.id = '{}_{} Iso: {}'.format(sp, seq.description, isoscore)
    seq.description = ''
    isotype_seqs[isotype].append(seq)

  print('finished')
  sys.stdout.flush()

processing Mus129S1_SvImJ_1509...Mus129S1_SvImJ_1509 tRNA file not found, skipping
processing MusAKR_J_1509...MusAKR_J_1509 tRNA file not found, skipping
processing MusA_J_1509...MusA_J_1509 tRNA file not found, skipping
processing MusBALB_cJ_1509...MusBALB_cJ_1509 tRNA file not found, skipping
processing MusC3H_HeJ_1509...MusC3H_HeJ_1509 tRNA file not found, skipping
processing MusC57BL_6NJ_1509...MusC57BL_6NJ_1509 tRNA file not found, skipping
processing MusCAROLI_EiJ_1509...MusCAROLI_EiJ_1509 tRNA file not found, skipping
processing MusCAST_EiJ_1509...MusCAST_EiJ_1509 tRNA file not found, skipping
processing MusCBA_J_1509...MusCBA_J_1509 tRNA file not found, skipping
processing MusDBA_2J_1509...MusDBA_2J_1509 tRNA file not found, skipping
processing MusFVB_NJ_1509...MusFVB_NJ_1509 tRNA file not found, skipping
processing MusLP_J_1509...MusLP_J_1509 tRNA file not found, skipping
processing MusNOD_ShiLtJ_1509...MusNOD_ShiLtJ_1509 tRNA file not found, skipping
processing MusNZO_HlLtJ_1

In [40]:
# Run this only after successfully running the above
for isotype in isotypes:
  fasta = 'fasta/euk-' + isotype + '.fa'
  if os.path.exists(fasta): # Force user to manually check whether it needs to be rewritten
    print("File exists: " + fasta)
    continue
  fasta_handle = open(fasta, 'w')
  SeqIO.write(isotype_seqs[isotype], fasta_handle, 'fasta')
  fasta_handle.close()

In [216]:
for isotype in isotypes:
  fasta = 'fasta/euk-' + isotype + '.fa'
  alignment = 'alignments/euk-' + isotype + '.sto'
  model = '/projects/lowelab/users/blin/tRNAscan/models/current/TRNAinf-euk.cm'
  # --ileaved ensures each sequence takes one line
  subprocess.call('cmalign -g --notrunc -o {} {} {}'.format(alignment, model, fasta), shell=True)

The parser requires the alignment to fit one line per sequence. Infernal splits it up when it's long with no user input, so we need to reconstruct the alignments one by one, with fully untruncated lines.

In [ ]:
for isotype in isotypes:
  alignment = 'alignments/euk-' + isotype + '.sto'
  lines = []
  ss_cons = ''
  rf = ''
  ordered_seqs = OrderedDict() # use ordered dict as an ordered set
  seqs = defaultdict(str)
  scores = defaultdict(str)
  
  for line in open(alignment):
    if line[0] == '\n' or line[0:4] in ['# ST', '#=GF', '#=GS']:
      lines.append(line)
    elif line == '//\n': continue
    elif line[0:7] == '#=GC SS':
      ss_cons += line.strip().split()[2]
      gaplength = line.count(' ') + 6
    elif line[0:7] == '#=GC RF':
      rf += line.strip().split()[2]
    # else, column is either nucleotide or score line for a sequence
    else:
      cols = line.strip().split()
      if cols[0] == "#=GR":
        scores[cols[1]] += cols[3]
      else:
        ordered_seqs[cols[0]] = ''
        seqs[cols[0]] += cols[1]
      
  for seq in ordered_seqs:
    lines.append('{}{}{}\n'.format(seq, ' ' * (gaplength - len(seq) + 5), seqs[seq]))
    lines.append('#=GR {}{}PP {}\n'.format(seq, ' ' * (gaplength - len(seq) - 3), scores[seq]))
  lines.append('#=GC SS_cons{}{}\n'.format(' ' * (gaplength - 7), ss_cons))
  lines.append('#=GC RF{}{}\n'.format(' ' * (gaplength - 2), rf))
  lines.append('//')
  
  outfile = open(alignment, 'w')
  for line in lines: outfile.write(line)
  outfile.close()

In [50]:
def position_base(positions, seq):
  for position_index, position in enumerate(positions):
    if position.paired:
      index1, index2 = position.position.split(':')
      index1, index2 = int(index1), int(index2)
      base_pair = "{}:{}".format(seq[index1 - 1], seq[index2 - 1])
      yield position.sprinzl, base_pair
    else:
      index = int(position.position)
      base = seq[index - 1]
      yield position.sprinzl, base
      
identities = pd.DataFrame()

for isotype in isotypes:
  sys.stdout.write('processing ' + isotype + '...')
  alignment = 'alignments/euk-' + isotype + '.sto'
  # get positions
  alignment_fhandle = open(alignment)
  positions = [] # list containing each position in the tRNA
  # first, get secondary structure
  for line in alignment_fhandle:
    if line[0:12] == '#=GC SS_cons':
      ss = line.strip().split()[-1]
  # parse secondary structure into regions and positions
  positions = annotate_positions(ss)
  # get counts for each position by parsing Stockholm file
  alignment_fhandle = open(alignment)
  trnas = []
  for line in alignment_fhandle:
    if line[0] in ["#", '\n', '/']: continue
    seqname, seq = line.strip().split()
    species = seqname.rsplit('_', 1)[0]
    if any(species_table.name == species): row = species_table[species_table.name == species]
    else: continue # skip species that need too much babying
    domain = row.domain.values[0]
    clade = row.clade.values[0]
    species_long = row.longname.values[0]
    trna = {'domain': domain, 'clade': clade, 'species': species, 'species_long': species_long, 'seqname': seqname, 'isotype': isotype}
    trna = {**trna, **{sprinzl: base for sprinzl, base in position_base(positions, seq)}}
    trnas.append(trna)
  identities = identities.append(trnas, ignore_index=True)
  print('done')

identities.fillna('.', inplace=True)

processing Ala...done
processing Arg...done
processing Asn...done
processing Asp...done
processing Cys...done
processing Gln...done
processing Glu...done
processing Gly...done
processing His...done
processing Ile...done
processing iMet...done
processing Leu...done
processing Lys...done
processing Met...done
processing Phe...done
processing Pro...done
processing Ser...done
processing Thr...done
processing Trp...done
processing Tyr...done
processing Val...done


## Additional sequence information

In [51]:
# Isotype, anticodon, score
seqinfo = []
for isotype in isotypes:
  alignment = 'alignments/euk-' + isotype + '.sto'
  for line in open(alignment):
    if line[0:4] != "#=GS": continue
    _, seqname, _, _, _, isotype, anticodon, _, _, _, score, _, isoscore = line.strip().split()
    seqinfo.append([seqname, isotype, anticodon[1:-1], float(score), float(isoscore)])
seqinfo = pd.DataFrame(seqinfo, columns=['seqname', 'isotype_ac', 'anticodon', 'score', 'isoscore'])
identities = identities.merge(seqinfo, on='seqname')

# GC content
paired_cols = identities.columns[list(map(lambda x: (':' in x), identities.columns))]
identities['GC'] = identities[paired_cols].apply(lambda x: sum((x == "G:C") | (x == "C:G"))/len(paired_cols), axis=1)

### Loop sizes

In [52]:
def bounds_to_cols(cols, start, end):
  selected_cols = []
  for col in cols:
    matches = re.findall('\d+', col)
    if len(matches) < 1: continue
    index = int(matches[0])
    if (index >= start and index <= end or col[0:3] == '{}i'.format(start - 1)) and col[0] != 'V':
      selected_cols.append(col)
  return selected_cols

dloop_cols = bounds_to_cols(identities.columns, 14, 21)
identities['D-loop'] = identities[dloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

acloop_cols = bounds_to_cols(identities.columns, 32, 38)
identities['AC-loop'] = identities[acloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

tpcloop_cols = bounds_to_cols(identities.columns, 54, 60)
identities['TPC-loop'] = identities[tpcloop_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

varm_cols = list(filter(lambda x: 'V' in x, identities.columns))
identities['V-arm'] = identities[varm_cols].apply(lambda x: len(x[(x != '.') & (x != '-')]), axis=1)

### Insertions/deletions

In [53]:
# Insertions
insertion_cols = list(filter(lambda x: re.match('\d+i', x), identities.columns))
identities['insertions'] = identities[insertion_cols].apply(lambda x: sum(x != '.'), axis=1)

# Deletions at positions that are not the variable arm
base_cols = list(filter(lambda x: re.match('^((\d+)|(\d+:\d+))$', x), identities.columns))
identities['deletions'] = identities[base_cols].apply(lambda x: ''.join(x).count('-'), axis=1)

### High-quality tRNA set

There was talk about using the high quality tRNAs to look at identity elements. Instead, I will just annotate specific tRNAs as high-quality or not. We can sort out the differences later.

In [54]:
# Import list of quality tRNAs
quality_trnas = [line.strip() for line in open('quality-set.out')]
identities['quality'] = identities.seqname.isin(quality_trnas)

### Restrict tRNAs by species

We may also want to limit the contribution of any single species, similar to how we built the isotype-specific models.

### Create single base columns from paired positions

This is mainly for plotting and such.

In [93]:
cols = list(filter(lambda x: ':' in x, identities.columns))
for col in cols:
  pos1, pos2 = col.split(':')
  base1 = [bases.split(':')[0] for bases in identities[col]]
  base2 = [bases.split(':')[1] for bases in identities[col]]
  identities[pos1] = base1
  identities[pos2] = base2

### Order columns

To make it look pretty.

In [94]:
def position_str_to_int(position):
  digits = re.findall('\d+', position)
  if len(digits) == 0: return -1
  insert = 0
  if 'i' in position and len(digits) == 2: insert = float(digits[1]) / 1000
  if position[0] == 'V':
    if ':' in position: return int(digits[0]) + 45 - 10 + insert # V11~V17
    else: return int(digits[0]) + 45 + 7 + insert # V1~V5
  if int(digits[0]) >= 46: return int(digits[0]) + 50 + insert # just add an arbitrarily large number to skip v-arm
  return int(digits[0]) + insert

identities = identities[sorted(list(identities.columns.values), key=position_str_to_int)]

In [95]:
identities.to_csv(path_or_buf='identities.tsv', sep='\t', index_label=False)

In [96]:
identities.head()

,clade,domain,isotype,seqname,species,species_long,isotype_ac,anticodon,score,isoscore,GC,D-loop,AC-loop,TPC-loop,V-arm,insertions,deletions,quality,0i1,0i2,0i3,0i4,0i5,0i6,0i7,0i8,0i9,0i10,0i11,1:72,1,1i1,2:71,2,2i1,3:70,3,3i1,3i2,3i3,3i4,3i5,3i6,3i7,4:69,4,4i1,4i2,4i3,4i4,4i5,4i6,4i7,4i8,4i9,4i10,4i11,5:68,5,5i1,5i2,5i3,5i4,5i5,5i6,5i7,6:67,6,6i1,7:66,7,7i1,7i2,7i3,7i4,7i5,7i6,7i7,7i8,7i9,7i10,7i11,7i12,7i13,7i14,8,8i1,8i2,8i3,8i4,9,9i1,9i2,9i3,10:25,10,10i1,11:24,11,12:23,12,12i1,12i2,12i3,12i4,13:22,13,13i1,14,14i1,14i2,14i3,14i4,14i5,14i6,14i7,14i8,14i9,14i10,14i11,14i12,14i13,14i14,14i15,14i16,14i17,14i18,14i19,14i20,14i21,14i22,14i23,14i24,14i25,15,16,16i1,16i2,16i3,16i4,16i5,16i6,16i7,16i8,16i9,16i10,16i11,16i12,16i13,16i14,16i15,16i16,16i17,16i18,16i19,16i20,17,17i1,17i2,17i3,18,19,20,20i1,20i2,20i3,20i4,20i5,20i6,20i7,20i8,20i9,20i10,21,21i1,22,22i1,23,23i1,24,24i1,25,25i1,25i2,25i3,25i4,25i5,25i6,25i7,25i8,25i9,25i10,25i11,25i12,25i13,26,26i1,26i2,26i3,26i4,27:43,27,27i1,27i2,27i3,27i4,27i5,28:42,28,28i1,28i2,28i3,28i4,29:41,29,29i1,30:40,30,30i1,31:39,31,32,33,34,35,35i1,36,37,37i1,37i2,37i3,37i4,37i5,37i6,37i7,37i8,37i9,37i10,37i11,37i12,37i13,37i14,37i15,37i16,37i17,37i18,37i19,37i20,37i21,37i22,37i23,37i24,37i25,37i26,37i27,37i28,37i29,37i30,37i31,37i32,37i33,37i34,37i35,37i36,37i37,37i38,37i39,37i40,37i41,37i42,37i43,37i44,37i45,37i46,37i47,37i48,37i49,37i50,37i51,37i52,37i53,37i54,37i55,37i56,37i57,37i58,37i59,37i60,37i61,37i62,37i63,37i64,37i65,37i66,37i67,37i68,37i69,37i70,37i71,37i72,37i73,37i74,37i75,37i76,37i77,37i78,37i79,37i80,37i81,37i82,37i83,37i84,37i85,37i86,37i87,37i88,37i89,37i90,37i91,37i92,37i93,37i94,37i95,37i96,37i97,37i98,37i99,37i100,37i101,37i102,37i103,37i104,37i105,37i106,37i107,37i108,37i109,37i110,37i111,37i112,37i113,37i114,37i115,37i116,37i117,37i118,37i119,37i120,37i121,37i122,37i123,37i124,37i125,37i126,37i127,37i128,37i129,37i130,37i131,37i132,37i133,37i134,37i135,37i136,37i137,37i138,37i139,37i140,37i141,37i142,37i143,37i144,37i145,37i146,37i147,37i148,37i149,37i150,37i151,37i152,37i153,37i154,37i155,37i156,37i157,37i158,37i159,37i160,37i161,37i162,37i163,37i164,37i165,37i166,37i167,37i168,37i169,37i170,37i171,37i172,37i173,37i174,37i175,37i176,37i177,37i178,37i179,37i180,37i181,37i182,37i183,37i184,37i185,37i186,37i187,37i188,37i189,37i190,37i191,37i192,37i193,37i194,37i195,37i196,37i197,37i198,37i199,37i200,37i201,37i202,37i203,37i204,37i205,37i206,37i207,37i208,37i209,37i210,37i211,37i212,37i213,37i214,37i215,37i216,37i217,37i218,37i219,37i220,37i221,37i222,37i223,37i224,37i225,37i226,37i227,37i228,37i229,37i230,37i231,37i232,37i233,37i234,37i235,37i236,37i237,37i238,37i239,37i240,37i241,37i242,37i243,37i244,37i245,37i246,37i247,37i248,37i249,37i250,37i251,37i252,37i253,37i254,37i255,37i256,37i257,37i258,37i259,37i260,37i261,37i262,37i263,37i264,37i265,37i266,37i267,37i268,37i269,37i270,37i271,37i272,37i273,37i274,37i275,37i276,37i277,37i278,37i279,37i280,37i281,37i282,37i283,37i284,37i285,37i286,37i287,37i288,37i289,37i290,37i291,37i292,37i293,37i294,37i295,37i296,37i297,37i298,37i299,37i300,37i301,37i302,37i303,37i304,37i305,37i306,37i307,37i308,37i309,37i310,37i311,37i312,37i313,37i314,37i315,37i316,37i317,37i318,37i319,37i320,37i321,37i322,37i323,37i324,37i325,37i326,37i327,37i328,37i329,37i330,37i331,37i332,37i333,37i334,37i335,37i336,37i337,37i338,37i339,37i340,37i341,37i342,37i343,37i344,37i345,37i346,37i347,37i348,37i349,37i350,37i351,37i352,37i353,37i354,37i355,37i356,37i357,37i358,37i359,37i360,37i361,37i362,37i363,37i364,37i365,37i366,37i367,37i368,37i369,37i370,37i371,37i372,37i373,37i374,37i375,37i376,37i377,37i378,37i379,37i380,37i381,37i382,38,38i1,39,39i1,40,40i1,40i2,40i3,40i4,41,41i1,42,42i1,43,44,44i1,44i2,44i3,44i4,44i5,44i6,44i7,44i8,44i9,44i10,44i11,44i12,44i13,44i14,44i15,44i16,44i17,44i18,44i19,44i20,44i21,44i22,44i23,45,V11:V21,V12:V22,V13:V23,V14:V24,V15:V25,V16:V26,V17:V27,V1,V2,V3,V4,V11,V12,V13,V14,V15,V16,V17,V21,V22,V23,V24,V25,V26,V27,46,47i1,47i